In [1]:
from global_parameters import GlobalParameters
from states import Key, Ciphertext
from decryption_circuit import decrypt_results
from verification_circuit import verify_deletion_counts
from utils import import_counts
from qiskit.circuit import qpy_serialization
import shutil

In [2]:
# Unzip files
shutil.unpack_archive("data_export.zip", "data")

In [3]:
# Import all files
with open("data/global_params.txt", "r") as params_file, open("data/key.txt", "r") as key_file, open("data/ciphertext.txt", "r") as ciphertext_file, open("data/message.txt", "r") as message_file:
    params = GlobalParameters.from_json(params_file.read())
    key = Key.from_json(key_file.read())
    ciphertext = Ciphertext.from_json(ciphertext_file.read(), "data/base_circuit.qpy")
    message = message_file.read()

In [4]:
# Test 1 - delete
deletion_counts_test1 = import_counts("data/test1-deletion-counts.csv")
verify_deletion_counts(deletion_counts_test1, key, params);

Accepted proof of deletion: 902/1000 (90.2%)
Rejected proof of deletion: 98/1000 (9.8%)
Out of the 98 rejected certificates, the following are the counts of the Hamming distances between the received certificate and the expected certificate:
Hamming distance 1: 95
Hamming distance 2: 3


In [5]:
# Test 2.1 - decrypt without error correction
decryption_counts_test2 = import_counts("data/test2-decryption-counts.csv")
decrypt_results(decryption_counts_test2, key, ciphertext, message)

# Test 2.2 - decrypt with error correction
print()
decrypt_results(decryption_counts_test2, key, ciphertext, message, error_correct=True)

Correct message decrypted: 867/1000 (86.7%)
Incorrect message decrypted: 133/1000 (13.3%)
Error detected during decryption process (hashes didn't match): 28/1000 (2.8000000000000003%)

Correct message decrypted: 976/1000 (97.6%)
Incorrect message decrypted: 24/1000 (2.4%)
Error detected during decryption process (hashes didn't match): 11/1000 (1.0999999999999999%)


In [8]:
# Test 3 - delete (possibly maliciously) and then decrypt
raw_counts_test3 = import_counts("data/test3-raw-counts.csv")
deletion_counts_test3 = import_counts("data/test3-deletion-counts.csv")
decryption_counts_test3 = import_counts("data/test3-decryption-counts.csv")
accepted_certificates_test3 = verify_deletion_counts(deletion_counts_test3, key, params)
# Test the decryption results for all the measurements, regardless of whether or not the certificate of deletion was accepted
decrypt_results(decryption_counts_test3, key, ciphertext, message)
print()
decrypt_results(decryption_counts_test3, key, ciphertext, message, error_correct=True)

# Test the decryption results for just the measurements where the certificate of deletion was accepted
accepted_deletion_decryption_counts_test3 = {}
for measurement, count in raw_counts_test3.items():
    deletion_measurement, decryption_measurement = measurement.split(" ")
    if deletion_measurement in accepted_certificates_test3:
        accepted_deletion_decryption_counts_test3[decryption_measurement] = accepted_deletion_decryption_counts_test3.get(decryption_measurement, 0) + count

print()
print("Of the measurements where the proof of deletion was accepted, the following are the decryption statistics:")
decrypt_results(accepted_deletion_decryption_counts_test3, key, ciphertext, message)

Accepted proof of deletion: 897/1000 (89.7%)
Rejected proof of deletion: 103/1000 (10.299999999999999%)
Out of the 103 rejected certificates, the following are the counts of the Hamming distances between the received certificate and the expected certificate:
Hamming distance 1: 97
Hamming distance 2: 6
Correct message decrypted: 66/1000 (6.6000000000000005%)
Incorrect message decrypted: 934/1000 (93.4%)
Error detected during decryption process (hashes didn't match): 848/1000 (84.8%)

Correct message decrypted: 134/1000 (13.4%)
Incorrect message decrypted: 866/1000 (86.6%)
Error detected during decryption process (hashes didn't match): 846/1000 (84.6%)

Of the measurements where the proof of deletion was accepted, the following are the decryption statistics:
Correct message decrypted: 63/897 (7.023411371237458%)
Incorrect message decrypted: 834/897 (92.97658862876254%)
Error detected during decryption process (hashes didn't match): 761/897 (84.83835005574136%)
